# 데이터 정제

### 전체 데이터셋에서 필요한 데이터 추출 및 어노테이션 수정
- 활용 데이터 셋 : hub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=153
- 어노테이션 형식 : json -> txt(Yolo)

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


### 라벨 및 원천 데이터 가져오기
- 경로 설정

In [2]:
# 원본 어노테이션이 있는 폴더 경로
origin_labels_path = "/content/drive/MyDrive/Colab Notebooks/Project/Chungbuk University/Capstone Design/AI_Model/PlantDiseaseDetection/data/raw/라벨링데이터/01.가지/0.정상/"
# 원본 이미지 데이터가 있는 폴더 경로
origin_images_path = "/content/drive/MyDrive/Colab Notebooks/Project/Chungbuk University/Capstone Design/AI_Model/PlantDiseaseDetection/data/raw/원천데이터/01.가지/0.정상/"

In [ ]:
# 데이터를 저장할 위치
labels_path = "/content/drive/MyDrive/Colab Notebooks/Project/Chungbuk University/Capstone Design/AI_Model/PlantDiseaseDetection/data/processed/labels/"
images_path = "/content/drive/MyDrive/Colab Notebooks/Project/Chungbuk University/Capstone Design/AI_Model/PlantDiseaseDetection/data/processed/images/"

In [4]:
from glob import glob
extension = "json"
labels_path_list = glob(f'{origin_labels_path}*.{extension}')
images_path_list = glob(f'{origin_images_path}*')

for i in labels_path_list:
  print(i)
for i in images_path_list:
  print(i)

/content/drive/MyDrive/Colab Notebooks/Project/Chungbuk University/Capstone Design/AI_Model/PlantDiseaseDetection/data/raw/라벨링데이터/01.가지/0.정상/V006_77_0_00_01_01_13_0_c03_20201209_0000_S01_1.jpg.json
/content/drive/MyDrive/Colab Notebooks/Project/Chungbuk University/Capstone Design/AI_Model/PlantDiseaseDetection/data/raw/라벨링데이터/01.가지/0.정상/V006_77_0_00_01_01_13_0_c03_20201209_0001_S01_1.jpg.json
/content/drive/MyDrive/Colab Notebooks/Project/Chungbuk University/Capstone Design/AI_Model/PlantDiseaseDetection/data/raw/라벨링데이터/01.가지/0.정상/V006_77_0_00_01_01_13_0_c03_20201209_0002_S01_1.jpg.json
/content/drive/MyDrive/Colab Notebooks/Project/Chungbuk University/Capstone Design/AI_Model/PlantDiseaseDetection/data/raw/라벨링데이터/01.가지/0.정상/V006_77_0_00_01_01_13_0_c03_20201209_0003_S01_1.jpg.json
/content/drive/MyDrive/Colab Notebooks/Project/Chungbuk University/Capstone Design/AI_Model/PlantDiseaseDetection/data/raw/라벨링데이터/01.가지/0.정상/V006_77_0_00_01_01_13_0_c03

### 데이터의 작물 부위 확인
- 사용하는 데이터셋에선 줄기 및 열매, 잎 등 여러가지 식물 부위의 데이터가 있다. 우리는 잎 감지를 위해 데이터를 사용하기 때문에 잎 사진 데이터를 확인해야된다.
- 예시)
  - 이미지 파일명: V006_77_0_00_01_01_13_0_c03_20201209_0000_S01_1.jpg
  - 어노테이션 파일명: V006_77_0_00_01_01_13_0_c03_20201209_0000_S01_1.jpg.json
    - 구분자 '_'로 토큰을 나누면 각각의 토큰에 의미를 부여할 수 있다. 이는 데이터 셋 생성자가 정해놓은 규칙으로 작성되어 있는데 중요한 토큰들만 살펴보자
      - token[1] : 세부과제
        - '77': 시설작물 질병, '78':노지작물 해충, '79' :노지작물 질병, '80': 과수화상병
      - token[2] : 데이터 종류
        - '0' : 정상 작물, '1': 질병, '2': 해충, '3':충해
      - token[4] : 작물 코드
        - 별첨 1
      - token[5] : 작물의 부위별 코드
        - 별첨 2

#### 별첨
##### 별첨 1 (식물 종류)
00 작물 없음
01 가지
02 고추
03 단호박
04 딸기
05 상추
06 수박
07 애호박
08 오이
09 쥬키니호박
10 참외
11 토마토
12 포도
##### 별첨 2 (식물 부위)
00 구분 없음
01 열매
02 꽃
03 잎
04 가지
05 줄기
06 뿌리
07 해충

In [5]:
import re

def is_area(filename,area_code):
    # '_'를 기준으로 분할하고 6번째 요소(식물 부위 코드) 확인 -> 해당 요소가 객체의 위치를 나타난다.
    token = filename.split('_')
    #print(parts)
    if len(token) >= 7:
        if token[5] == area_code: # 03번은 잎이다.
            return True
        else:
            return False
    else:
        return False

### 데이터 어노테이션 형식 변환
- 기존 데이터셋의 어노테이션은 json형식의 커스텀 어노테이션 형식을 고수한다. 우리는 YOLO모델을 사용하여 학습시키기때문에 YOLO 어노테이션(txt)형식으로 변환해준다.
  - Yolo 어노테이션 형식
    - ex) class_code x_center y_center box_width box_height

In [6]:
def convert_to_yolo_annotation(path):
    # JSON 파일 읽기
    with open(path, 'r') as f:
        json_data = json.load(f)
        json_string = json.dumps(json_data)
        data = json.loads(json_string)
        # 이미지 너비와 높이
        image_width = data["description"]["width"]
        image_height = data["description"]["height"]

        # 바운딩 박스 좌표
        points = data["annotations"]["points"][0]

    # center구하고 데이터 정규화
    x_center = (points["xtl"] + points["xbr"]) / 2 / image_width
    y_center = (points["ytl"] + points["ybr"]) / 2 / image_height
    box_width = (points["xbr"] - points["xtl"]) / image_width
    box_height = (points["ybr"] - points["ytl"]) / image_height

    # YOLO 어노테이션 형식 리스트 반환
    return [x_center, y_center, box_width, box_height]

### 변환한 어노테이션 파일 저장
- 학습에 사용하기 위해 새로 생성한 어노테이션을 Yolo train 폴더 구조에 맞게 저장해야한다.
- Yolo train 폴더 구조
  - data <br>
  ㄴ trian/<br>
  &nbsp;&nbsp;&nbsp; 1.jpg(이미지 파일)<br>
  &nbsp;&nbsp;&nbsp; 1.txt(어노테이션 파일)<br>
  ㄴ vaild/<br>
  &nbsp;&nbsp;&nbsp; 2.jpg(이미지 파일)<br>
  &nbsp;&nbsp;&nbsp; 2.txt(어노테이션 파일)<br>
  ㄴ test/<br>
  &nbsp;&nbsp;&nbsp; 3.jpg(이미지 파일)<br>
  &nbsp;&nbsp;&nbsp; 3.txt(어노테이션 파일)<br>
   .yaml<br>

In [ ]:
import os
import json

def store(path,basename,data):
    # Check if the destination folder exists, if not, create it
    result = []
    output_folder = path
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        print(f"Destination folder '{output_folder}' created.")
    bbox_string = " ".join([str(x) for x in data])
    result.append(f"0 {bbox_string}")
    print("label_ok")
    if result:
      basename = basename.split(".")[0]
      with open(os.path.join(output_folder, f"{basename}.txt"), "w", encoding="utf-8") as f:
          f.write("\n".join(result))


In [ ]:
import os
import shutil

def copy_images(source_path, destination_folder):
  if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)
    print(f"Destination folder '{destination_folder}' created.")

  filename = os.path.basename(source_path)

  destination_path = os.path.join(destination_folder, filename)

  try:
      # Copy the file
      shutil.copy2(source_path, destination_path)
      print("image_ok")
  except Exception as e:
      print(f"Error occurred while copying the file: {e}")


In [ ]:
from glob import glob
extension = "json"
labels_path_list = glob(f'{origin_labels_path}*.{extension}')
print(labels_path_list)
images_path_list = glob(f'{origin_images_path}*')

for i in labels_path_list:
    basename= os.path.basename(i)
    if is_leaf(basename):
        store_json(labels_path,basename,convert_to_yolo_annotation(i))

for i in images_path_list:
    basename= os.path.basename(i)
    if is_leaf(basename):
        copy_images(i,images_path)

['/content/drive/MyDrive/Colab Notebooks/Project/Chungbuk University/Capstone Design/AI_Model/PlantDiseaseDetection/data/raw/라벨링데이터/01.가지/0.정상/V006_77_0_00_01_01_13_0_c03_20201209_0000_S01_1.jpg.json', '/content/drive/MyDrive/Colab Notebooks/Project/Chungbuk University/Capstone Design/AI_Model/PlantDiseaseDetection/data/raw/라벨링데이터/01.가지/0.정상/V006_77_0_00_01_01_13_0_c03_20201209_0001_S01_1.jpg.json', '/content/drive/MyDrive/Colab Notebooks/Project/Chungbuk University/Capstone Design/AI_Model/PlantDiseaseDetection/data/raw/라벨링데이터/01.가지/0.정상/V006_77_0_00_01_01_13_0_c03_20201209_0002_S01_1.jpg.json', '/content/drive/MyDrive/Colab Notebooks/Project/Chungbuk University/Capstone Design/AI_Model/PlantDiseaseDetection/data/raw/라벨링데이터/01.가지/0.정상/V006_77_0_00_01_01_13_0_c03_20201209_0003_S01_1.jpg.json', '/content/drive/MyDrive/Colab Notebooks/Project/Chungbuk University/Capstone Design/AI_Model/PlantDiseaseDetection/data/raw/라벨링데이터/01.가지/0.정상/V006_77_0_00_